# Simulation of lifetimes for Schmidt distributions

In [1]:
import numpy as np
import os, sys, yaml, re
from IPython.display import display, Markdown, Latex #Can write latex too!!!!

In [2]:
class DecayChain(yaml.YAMLObject):
    
    yaml_tag = u'!DecayChain'
    
    def __init__(self, idd="", be="", pi="", ie="", it="", ea=[], eas=[], ta=[]):
        self.ID = idd
        self.BeamEnergy = be
        self.Pixel = pi
        self.ImplantEnergy = ie
        self.ImplantTime = it
        self.EAlpha, self.EAlphaSigma, self.TAlpha = ea, eas, ta

class SetDecayChains:
    
    def __init__(self, path='', ids=[]):
        s_files = " ".join(os.listdir(path))
        files = []
        for s in ids:
            files += (sorted(re.findall(string=s_files, pattern="Chain"+s+"\d+.yml")))
        print("Reading the following files:", files)
        self.Chains = []
        for f in files:
            f_in = open(path+f, 'r')
            self.Chains.append(yaml.load(f_in))
            f_in.close()

In [3]:
s_path = "E115_Chains/"
s_id = ["14", "11", "17"]

setDC = SetDecayChains(s_path, s_id)
max_steps = 0
for chain in setDC.Chains:
    if len(chain.TAlpha) > max_steps:
        max_steps = len(chain.TAlpha)
print("Max steps=", max_steps)
times = np.zeros((len(setDC.Chains), max_steps))
for row, chain in enumerate(setDC.Chains):
    steps = len(chain.TAlpha)
    for i in range(max_steps):
        if i < steps:
            times[row][i] = chain.TAlpha[i]
        else:
            times[row][i] = np.nan       
times

Reading the following files: ['Chain1401.yml', 'Chain1402.yml', 'Chain1403.yml', 'Chain1404.yml', 'Chain1405.yml', 'Chain1406.yml', 'Chain1407.yml', 'Chain1101.yml', 'Chain1102.yml', 'Chain1103.yml', 'Chain1104.yml', 'Chain1701.yml', 'Chain1702.yml', 'Chain1703.yml']
Max steps= 3


array([[  2.27000000e-01,   3.78000000e-01,              nan],
       [  6.45000000e-02,   3.66000000e-01,              nan],
       [  2.61000000e-01,   1.15000000e+00,   3.43000000e-01],
       [  1.46000000e+00,   2.62000000e-02,   4.32000000e-01],
       [  3.45000000e-01,   3.69000000e-01,   1.44000000e+01],
       [  2.10000000e-01,   1.05000000e+00,   8.27000000e+00],
       [  8.15000000e-01,   2.33000000e+00,   2.89000000e+00],
       [  2.56200000e-01,   1.40270000e+00,   1.97750000e+00],
       [  6.61000000e-02,   1.55000000e+00,   2.36380000e+00],
       [  2.35070000e+00,   2.25822000e+01,   6.01855000e+01],
       [  5.36000000e-02,   4.67100000e-01,   9.08000000e-02],
       [  2.14000000e-01,   1.54000000e+00,   7.57000000e+00],
       [  5.91000000e-02,   8.24000000e-01,              nan],
       [  4.55000000e-02,   1.42000000e-02,              nan]])

## Sorting out chains of different lengths into arrays

Need to simulate lifetime values for the number of chains we have in the set for enough number of times. To simplify, maybe it would be nice to reshuffle the set into two subsets depending on the length.

In [4]:
N_j = np.count_nonzero(~np.isnan(times), axis=1)
N_j

array([2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2])

In [5]:
ind1 = np.argwhere(N_j == max_steps)[:,0]
ind1

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [6]:
ind2 = np.argwhere(N_j == max_steps-1)[:,0]
ind2

array([ 0,  1, 12, 13])

In [7]:
times1 = times[ind1]
times1

array([[  2.61000000e-01,   1.15000000e+00,   3.43000000e-01],
       [  1.46000000e+00,   2.62000000e-02,   4.32000000e-01],
       [  3.45000000e-01,   3.69000000e-01,   1.44000000e+01],
       [  2.10000000e-01,   1.05000000e+00,   8.27000000e+00],
       [  8.15000000e-01,   2.33000000e+00,   2.89000000e+00],
       [  2.56200000e-01,   1.40270000e+00,   1.97750000e+00],
       [  6.61000000e-02,   1.55000000e+00,   2.36380000e+00],
       [  2.35070000e+00,   2.25822000e+01,   6.01855000e+01],
       [  5.36000000e-02,   4.67100000e-01,   9.08000000e-02],
       [  2.14000000e-01,   1.54000000e+00,   7.57000000e+00]])

In [8]:
times2 = times[ind2][:, :-1]
times2

array([[ 0.227 ,  0.378 ],
       [ 0.0645,  0.366 ],
       [ 0.0591,  0.824 ],
       [ 0.0455,  0.0142]])

Well, these additional steps may only confuse the reader!!! For the geometrical mean with nans I can implement something in cython. 

## Sample from exponential distribution

In [9]:
nbr_sets = 10
shape = (nbr_sets, *np.shape(times))
sim = np.random.exponential(scale=1, size=shape)
np.shape(sim)

(10, 14, 3)

### Masking to insert the nans

In [10]:
m2d = np.ma.masked_array(np.isnan(times))
m2d

masked_array(data =
 [[False False  True]
 [False False  True]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False False]
 [False False  True]
 [False False  True]],
             mask =
 False,
       fill_value = True)

In [11]:
mask3d = np.ma.dstack([m2d]*nbr_sets)
np.shape(mask3d)

(14, 3, 10)

In [12]:
sim[mask3d]

ValueError: operands could not be broadcast together with shapes (10,14,3) (14,3,10) 

### Sooooo much easier

Short chains data set duplicated

In [13]:
def g_nan_mean(data):
    ret = np.empty(np.shape(data)[0])
    for i in range(np.shape(data)[0]):
        temp = 1
        steps = 0
        for j in range(np.shape(data)[1]):
            if ~np.isnan(data[i,j]):
                temp *= data[i,j]
            else:
                break
            steps += 1
        ret[i] = temp**(1./steps)
    return ret

In [14]:
nbr_sets = 2
shape = (nbr_sets, *np.shape(times))
sim = np.empty(shape)
for i in range(nbr_sets):
    sim[i] = times
sim[0, :, 0], sim

(array([ 0.227 ,  0.0645,  0.261 ,  1.46  ,  0.345 ,  0.21  ,  0.815 ,
         0.2562,  0.0661,  2.3507,  0.0536,  0.214 ,  0.0591,  0.0455]),
 array([[[  2.27000000e-01,   3.78000000e-01,              nan],
         [  6.45000000e-02,   3.66000000e-01,              nan],
         [  2.61000000e-01,   1.15000000e+00,   3.43000000e-01],
         [  1.46000000e+00,   2.62000000e-02,   4.32000000e-01],
         [  3.45000000e-01,   3.69000000e-01,   1.44000000e+01],
         [  2.10000000e-01,   1.05000000e+00,   8.27000000e+00],
         [  8.15000000e-01,   2.33000000e+00,   2.89000000e+00],
         [  2.56200000e-01,   1.40270000e+00,   1.97750000e+00],
         [  6.61000000e-02,   1.55000000e+00,   2.36380000e+00],
         [  2.35070000e+00,   2.25822000e+01,   6.01855000e+01],
         [  5.36000000e-02,   4.67100000e-01,   9.08000000e-02],
         [  2.14000000e-01,   1.54000000e+00,   7.57000000e+00],
         [  5.91000000e-02,   8.24000000e-01,              nan],
         [ 

Simulated chains

In [15]:
nbr_sets = 30000
shape = (nbr_sets, *np.shape(times))
sim = np.random.exponential(scale=1, size=shape)
for i in range(nbr_sets):
    sim[i][np.isnan(times)] = np.nan
sim

array([[[  6.14678440e-01,   6.70126764e-01,              nan],
        [  3.40053986e-02,   8.22395282e-01,              nan],
        [  8.03856992e-01,   5.77115912e-01,   1.35687003e-01],
        ..., 
        [  3.51631776e-01,   2.44450327e+00,   1.35385812e+00],
        [  7.16541237e-01,   1.74881141e+00,              nan],
        [  1.68238359e-04,   9.86797818e-01,              nan]],

       [[  4.08016313e-01,   2.20555277e-01,              nan],
        [  6.50434323e-02,   2.28892836e-01,              nan],
        [  9.41885510e-02,   3.49063474e+00,   9.09514174e-01],
        ..., 
        [  4.28603400e-01,   2.49660559e+00,   2.74070404e+00],
        [  1.72839191e+00,   1.07022681e-01,              nan],
        [  1.16102622e+00,   3.42278347e+00,              nan]],

       [[  1.34881213e+00,   6.09297376e-01,              nan],
        [  1.32447293e+00,   9.50221051e-01,              nan],
        [  3.85982074e+00,   2.25543466e+00,   2.26560647e+00],
        

In [16]:
theta = np.log(sim)
theta_var = np.zeros(np.shape(theta))
gen_Schmidt = np.zeros(np.shape(theta)[:-1])
for i in range(nbr_sets):
    theta_var[i] = np.square(theta[i] - np.nanmean(theta[i], axis=0))
    gen_Schmidt[i] = g_nan_mean(theta_var[i])

#theta_var
gen_Schmidt = np.sqrt(np.mean(gen_Schmidt, axis=1))
print(gen_Schmidt)

[ 1.0564578   1.05680423  1.08838862 ...,  0.94250492  0.92210166
  0.80971526]


### Expected generalised Schmidt value

In [17]:
E_Schmidt = np.mean(gen_Schmidt)
E_Schmidt

0.8611277569069522

Percentiles

In [18]:
np.percentile(gen_Schmidt, q=5), np.percentile(gen_Schmidt, q=95)

(0.62693873367090591, 1.1338703581954228)